In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,411 kB]
Hit:12 http://ppa.launchpad.net

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-16 11:29:55--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-05-16 11:29:56 (6.10 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL_AWS_software").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Configuration for RDS instance
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_db"
mode_1="append"
config_1 = {"user":"postgres",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}
mode_2="overwrite"
config_2 = {"user":"postgres",
          "password": "<insert password>",
          "truncate": "true",
          "driver":"org.postgresql.Driver"}

In [ ]:
from pyspark import SparkFiles
#load software reviews data from s3.amazonaws.com
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

software_df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Digital_Software_v1_00.tsv.gz"), inferSchema=True, dateFormat="yyyy-MM-dd",sep="\t")
software_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17747349|R2EI7QLPK4LF7U|B00U7LCE6A|     106182406|CCleaner Free [Do...|Digital_Software|          4|            0|          0|   N|                Y|          Four Stars|      So far so good| 2015-08-31|
|         US|   10956619|R1W5OMFK1Q3I3O|B00HRJMOM4|     162269768|ResumeMaker Profe...|Digital_Software|          3|    

# **Drop duplicates and incomplete rows**

In [ ]:
print(software_df.count())
software_df = software_df.dropna()
print(software_df.count())
software_df = software_df.dropDuplicates()
print(software_df.count())

102084
102077
102077


# **Examine the schema**

In [ ]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



# **Transfor Dataframe type**

In [ ]:
import datetime
software_df = software_df.withColumn("review_date",software_df["review_date"].cast("date"))
software_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   38713083|R1R421TU4E484F|B00FJIBYLW|     880437537|The Print Shop De...|Digital_Software|          2|            0|          1|   N|                Y|Even though this ...|Even though this ...| 2015-08-30|
|         US|   32002607|R25BMAAMJFM3G6|B003LJXELE|     527297648|Typing Instructor...|Digital_Software|          1|    

In [ ]:
software_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



# **Create Dataframe for review_id_table**

In [ ]:
review_id_df = software_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1R421TU4E484F|   38713083|B00FJIBYLW|     880437537| 2015-08-30|
|R25BMAAMJFM3G6|   32002607|B003LJXELE|     527297648| 2015-08-29|
|R3QT4NP1EY53YR|   13303537|B00H9A60O4|     608720080| 2015-08-28|
| R1Q8XBH55CII3|   46490624|B00NG7JQHW|      42668324| 2015-08-25|
|R2MM3GBNRQ0H4N|   31207284|B00NG7JVSQ|     811978073| 2015-08-24|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
review_id_df_pre = spark.read.jdbc(url=jdbc_url,table="review_id_table", properties=config_1)
review_id_df = review_id_df.join(review_id_df_pre, ["review_id"], "leftanti")
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1039AWAAHWVDQ|   45648423|B008S0IP38|     383426331| 2012-11-06|
|R103AF1E7F9HDL|   19545114|B00E7X8YK4|     713752890| 2014-08-09|
|R10ED5FSM3SZGU|   23956042|B00FFINOWS|     875090538| 2014-02-21|
|R10GS4J96PML2E|   10729322|B00MHZ6Z64|     249773946| 2015-02-14|
|R10O1VRSXC1V0I|   39675928|B009HBCZPQ|     808553197| 2013-02-19|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
print(review_id_df.count())
review_id_df = review_id_df.dropDuplicates(["review_id"])
print(review_id_df.count())

102077
102077


# **Create Dataframe for products_table**

In [ ]:
products_df = software_df.select(["product_id", "product_title"])
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FJIBYLW|The Print Shop De...|
|B003LJXELE|Typing Instructor...|
|B00H9A60O4|Avast Free Antivi...|
|B00NG7JQHW|TurboTax Business...|
|B00NG7JVSQ|TurboTax Deluxe F...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
products_df_pre = spark.read.jdbc(url=jdbc_url,table="products", properties=config_1)
products_df = products_df.join(products_df_pre, ["product_id"], "leftanti")
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007QV9EXU|Mavis Beacon Keyb...|
|B007QV9EXU|Mavis Beacon Keyb...|
|B007QV9EXU|Mavis Beacon Keyb...|
|B007QV9EXU|Mavis Beacon Keyb...|
|B007QV9EXU|Mavis Beacon Keyb...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
print(products_df.count())
products_df = products_df.dropDuplicates(["product_id"])
print(products_df.count())

102077
2998


# **Create Dataframe for customers_table**

In [ ]:
customers_df_new = software_df.select(["customer_id"])
customers_df_new = customers_df_new.groupBy("customer_id").count()
customers_df_new.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    8429442|    1|
|   15259970|    1|
|   49572061|    1|
|   49007721|    1|
|   47356179|    1|
+-----------+-----+
only showing top 5 rows



In [ ]:
# Creat a copy of sql database
customers_df_pre = spark.read.jdbc(url=jdbc_url,table="customers", properties=config_1)
schema = customers_df_pre.schema
customers_df_pre_pd = customers_df_pre.toPandas()
_customers_df_pre = spark.createDataFrame(customers_df_pre_pd,schema=schema)
del customers_df_pre_pd

In [ ]:
# Get the total counts of each customer
if _customers_df_pre.count():
  customers_df=customers_df_new.join(_customers_df_pre,on=["customer_id"],how="outer")
  customers_df=customers_df.fillna(0)
  customers_df=customers_df.withColumn('total', sum(customers_df[col] for col in ["count", "customer_count"]))
  customers_df = customers_df.drop("count", "customer_count")
  customers_df = customers_df.withColumnRenamed("total", "customer_count")
  customers_df.show(5)
else:
  customers_df = customers_df_new.withColumnRenamed("count", "customer_count")
  customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    8429442|             1|
|   15259970|             1|
|   49572061|             1|
|   49007721|             1|
|   47356179|             1|
+-----------+--------------+
only showing top 5 rows



In [ ]:
print(customers_df.count())

94308


# **Create Dataframe for vine_table**

In [ ]:
vine_df = software_df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1R421TU4E484F|          2|            0|          1|   N|
|R25BMAAMJFM3G6|          1|            0|          0|   N|
|R3QT4NP1EY53YR|          3|            0|          0|   N|
| R1Q8XBH55CII3|          5|            0|          0|   N|
|R2MM3GBNRQ0H4N|          4|            0|          3|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Drop duplicated review existed in sql database
vine_df_pre = spark.read.jdbc(url=jdbc_url,table="vine_table", properties=config_1)
vine_df = vine_df.join(vine_df_pre, ["review_id"], "leftanti")
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1039AWAAHWVDQ|          4|            1|          4|   N|
|R103AF1E7F9HDL|          3|            0|          0|   N|
|R10ED5FSM3SZGU|          1|            1|          3|   N|
|R10GS4J96PML2E|          5|            1|          1|   N|
|R10O1VRSXC1V0I|          4|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
print(vine_df.count())
vine_df = vine_df.dropDuplicates(["review_id"])
print(vine_df.count())

102077
102077


# **Write DataFrame to RDS**

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode_1, properties=config_1)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode_1, properties=config_1)

In [ ]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode_2, properties=config_2)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode_1, properties=config_1)